<a href="https://colab.research.google.com/github/b3GOR/Flang/blob/main/FLang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка библиотек и модели


In [ ]:
%%capture
!pip install transformers accelerate

In [ ]:
# Загрузка необходимых библиотек
from transformers import AutoTokenizer
import transformers
import torch
from google.colab import files

In [ ]:
# Загрузка модели
model = "b3G0R/FLang"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Загрузка пользовательского словаря

In [ ]:
# Загрузка словаря пользователя из слов, которые он уже знает
uploaded = files.upload()
file_names = list(uploaded.keys())
first_file_name = file_names[0]
with open(first_file_name, 'r') as file:
  w = file.read().split()
  dct = ', '.join(w)

# Генератор промтов, функция скорриноговой модели

In [ ]:
# Массив промптов для генерации нового промпта в резульатате комбинации
v = ['Write a text on English of {sentence} sentence about {topic} topic, but use these words:{words}.',

'Create a passage in English with {sentence} sentence(s) discussing the topic of {topic}. Make sure to include the words:{words}.',

'Compose an English text comprising {sentence} sentence(s) that revolves around the subject of {topic}. Incorporate the words:{words} into your writing.',

'Craft a piece of writing in English that consists of {sentence} sentence(s) and centers on the topic of {topic}. Ensure you incorporate the words:{words} into your composition.',

'Write a short English essay with {sentence} sentence(s) addressing the {topic} topic. You must utilize the words:{words} in your writing.',

'Develop an English narrative containing {sentence} sentence(s) exploring the subject of {topic}. Be sure to include the words:{words} in your narrative.',

'Compose a paragraph in English consisting of {sentence} sentence(s) that discusses the {topic} topic. Make sure to use the words:{words} in your paragraph.',

'Create an English composition with {sentence} sentence(s) that delves into the subject of {topic}. Incorporate the words:{words} into your composition.',

'Write an English text with {sentence} sentence(s) focusing on the {topic} topic. Your text should feature the words:{words}.',

'Craft an English essay comprising {sentence} sentence(s) that explores the {topic} topic. Make sure to include the words:{words} in your essay.']

In [ ]:
# Генерирование stopwords:

# stopwords spacy
import spacy
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
stopwords_spacy = set(en.Defaults.stop_words)

# stopwords nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_nltk = set(stopwords.words('english'))

# stopwords sklearn
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stopwords_sklearn = set(ENGLISH_STOP_WORDS)

# stopwords gensim
from gensim.parsing.preprocessing import STOPWORDS
stopwords_gensim = set(STOPWORDS)

# объединим стоп-слова
stopwords_all = stopwords_gensim.union(stopwords_sklearn, stopwords_nltk, stopwords_spacy)

# Создадим множество из лексем стоп-слов через spacy (на всякий случай)
nlp = spacy.load('en_core_web_sm')
stopwords_all_lex = set() # Создадим множество из лексем стоп-слов через spacy (на всякий случай)
for _ in stopwords_all:
  stopwords_all_lex.add(str(nlp(_)).lower())

# Добавим знаки препинания в stopwords_all_lex
stopwords_all_lex.update((".", ",", ";", "'", "...", "(", ")", "[", "]", ":", "-", "?", "!", "\""))

In [ ]:
# Создадим множество dct_set и добавим туда токены из dct (кроме запятых)
dct_set = set()
for _ in dct.split(","):
  dct_set.add(str(nlp(_.strip())).lower())

In [ ]:
# Генерация промпта на основе словаря промптов путем их случайной комбинации
import numpy as np

d = np.array(v)
def generate_prompt():
    start = ['Create an English','Compose an English text','Craft an English','Write a text on English','Develop an English']
    middle = ['{sentence}', '{topic}', 'but use these words:{words}']
    words = np.random.permutation(len(d))[:1]
    target_word = d[np.random.randint(0, len(d))]
    return f"{start[np.random.randint(0, len(start))]} - {words} - {middle[np.random.randint(0, len(middle))]} - {target_word}"

d =generate_prompt().split('-')[-1].strip()
print(d)

# Генератор текста к словам

In [ ]:
# @title Генератор текста к словам
topic = "" # @param {type:"string"}
sentences = "" # @param {type:"string"}
level = "A1" # @param ["A1", "A2", "B1", "B2", "C1", "C2"]
words = "" # @param {type:"string"}

r = d.replace('{sentence}', sentences).replace('{topic}', topic).replace('{words}', words)
prompt = r
# Системный промпт и обычный промпт
SYS = f'''You are an English teacher, and your task is to compose a well-written English text that demonstrates strong grammar, style, and semantics.
The text should be tailored to an English proficiency level of {level}.
The text must consist of only these words ({dct}) and these words {words}.'''
# Добавление в словарь уже изученных слов, новые слова, которые ввел пользователь
dct_n = dct + ', ' + words
del  dct
dct = dct_n
del dct_n


####### Создадим множество words_set и добавим туда токены из words. Эти же токены
####### добавим в dct_set

word_set = set()
for _ in words.split(","):
  word_set.add(str(nlp(_.strip())).lower())
  dct_set.add(str(nlp(_.strip())).lower())
#######
# Генерация текста
sequences = pipeline(
    f'[INST] <<SYS>> {SYS}  <</SYS>> {prompt} [/INST]',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=500,
)

generated_text = sequences[0]['generated_text']

# Удаление информации о введенном промпте (на выходе только текст)
while "[INST]" in generated_text:
    start_index = generated_text.find("[INST]")
    end_index = generated_text.find("[/INST]", start_index)

    if start_index != -1 and end_index != -1:
        generated_text = generated_text[:start_index] + generated_text[end_index + 7:]
    else:
        break

# Разделяем предложения символом новой строки
generated_text = generated_text.replace(".", ".\n")

print(generated_text)

# Скоринговая модель

In [ ]:
######
###### Функция reward_scoring показывает долю токенов
###### из dct_set(words_study) и stopwords_all_lex(stopwords_set) в сгенерированной
###### фразе. В случае, если во фразе отсутствуют слова из word_set, возвращаемое значение
###### функции reward_scoring домножается на 0.1 ** (число отсутствующих слов из word_set во фразе)

######
def reward_scoring(sentence: str, target_word: set,
                   words_study: set, stopwords_set:set) -> float:
  sentence_lex = []
  doc = nlp(sentence)
  for token in doc:
    sentence_lex.append(str(token.lemma_).lower())

  count_words_in_sets = 0 # число слов в множествах words_study и stopwords
  for _ in sentence_lex:
    if _ in words_study or _ in stopwords_set:
      count_words_in_sets += 1
  rew_score = count_words_in_sets/len(sentence_lex)

  count_target_word = 0 # проверяем, сколько таргет-слов есть в sentence
  for _ in target_word:
    if _ in sentence_lex:
      count_target_word += 1

  rew_score *= (0.1 ** (len(target_word) - count_target_word))
  return rew_score

In [ ]:
###### Расчет rew_score

sentence = "The dog is sitting near my daughter." ### Введите в кавычках сгенерированную фразу
reward_scoring(sentence, word_set, dct_set, stopwords_all_lex)

0.08750000000000001